In [ ]:
from Functions.documents import load_documents
from Functions.text import clear_text, remove_stopwords, split_text_into_chunks, remove_portuguese_accents



DOCUMENT_PATHS = ["./Documents/a (1).pdf", "./Documents/FAQ (1).pdf"]
CHUNK_SIZE = 500



# BEGIN = Processing documents 
texts = load_documents(DOCUMENT_PATHS)
cleaned_texts = [clear_text(text) for text in texts]
no_stopwords_text = [remove_stopwords(text) for text in cleaned_texts]
no_accents_portuguese = [remove_portuguese_accents(text) for text in no_stopwords_text]
joined_texts = " ".join(no_accents_portuguese)
joined_texts = joined_texts.lower()
text_chunks = split_text_into_chunks(joined_texts, chunk_size=CHUNK_SIZE)
# END = Processing documents 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
len(text_chunks)

22

In [4]:
len(text_chunks[0])

3894

In [8]:
from sentence_transformers import SentenceTransformer, util


c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


import gym
import numpy as np
from gym import spaces
from sentence_transformers import SentenceTransformer, util
from Functions.documents import load_documents
from Functions.text import clear_text, remove_stopwords, split_text_into_chunks

In [9]:
model = SentenceTransformer("all-MiniLM-L6-v2")

c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
encoded = model.encode(text_chunks[0])

In [11]:
not_encoded = text_chunks[0]

In [12]:
len(encoded)

384

In [13]:
len(not_encoded)

3894

In [14]:
document_embeddings = [model.encode(chunk) for chunk in text_chunks] 

In [22]:
question_embedding = model.encode("Como e quando posso realizar o trancamento de matrícula?")

In [24]:
from sentence_transformers import util

# Compute cosine similarity between the question and all document chunks
cosine_similarities = [util.pytorch_cos_sim(question_embedding, doc_emb).item() for doc_emb in document_embeddings]

# Convert cosine similarity to a distance measure (1 - similarity)
cosine_distances = [1 - sim for sim in cosine_similarities]

print(cosine_distances)  # List of distances (lower means more similar)

[0.5742928385734558, 0.530009925365448, 0.4637027382850647, 0.6362056136131287, 0.5164728164672852, 0.5483729243278503, 0.5802350044250488, 0.6583434045314789, 0.6417689025402069, 0.5609636306762695, 0.6662200391292572, 0.6107828915119171, 0.6246375143527985, 0.6299183368682861, 0.5671376883983612, 0.526781439781189, 0.6537935137748718, 0.5203404426574707, 0.6117911636829376, 0.6069388389587402, 0.5793417096138, 0.5098774135112762]


In [25]:
# Find the index of the most similar (highest similarity) and least similar (lowest similarity) document chunks
max_index = max(range(len(cosine_similarities)), key=lambda i: cosine_similarities[i])
min_index = min(range(len(cosine_similarities)), key=lambda i: cosine_similarities[i])

print(f"Most relevant document chunk index: {max_index} (Score: {cosine_similarities[max_index]})")
print(f"Least relevant document chunk index: {min_index} (Score: {cosine_similarities[min_index]})")


Most relevant document chunk index: 2 (Score: 0.5362972617149353)
Least relevant document chunk index: 10 (Score: 0.3337799608707428)


In [28]:
print(text_chunks[2])

x disposicoes finais transitorias 43 glossario 45 8 capitulo i matricula secao i concessao matricula art 1º matricula concedida i candidato classificado processo seletivo adotado universidade periodo letivo obteve classificacao reclassificacao ii estudante credenciado convenio instituicoes nacionais estrangeiras convenio intercambio acordo cultural brasil outros paises iii estudante admitido estudante especial iv estudante transferido ex officio v estudante transferido cursos ufba outras ies secao ii efetivacao matricula art 2º candidato procurador legalmente constituido efetuara matricula local periodo fixados edital especifico convocacao classificados processo seletivo sob pena perda direito art 3º matricula compreendera i apresentacao entrega documentos ii inscricao componentes curriculares curriculo curso 1º documentacao incompleta dara direito matricula universidade 2º oriundo pais estrangeiro certificado conclusao diploma graduacao pos graduacao historico escolar equivalente deve

In [39]:
import textdistance
# https://github.com/life4/textdistance?tab=readme-ov-file

f1 = "o carro está andando"
# f2 = "o ônibus está em movimento"
f2 = "o ônibus andando"
print("monge: ",textdistance.monge_elkan(f1,f2))
print("cosine: ",textdistance.cosine(f1,f2))
print("hamming: ",textdistance.hamming.normalized_similarity(f1,f2))

monge:  0.034999999999999996
cosine:  0.6149186938124421
hamming:  0.09999999999999998


In [ ]:

# Compute cosine similarity between the question and all document chunks
cosine_similarities = [util.pytorch_cos_sim(self.query_embedding, doc_emb).item() for doc_emb in self.document_embeddings]

# Convert cosine similarity to a distance measure (1 - similarity)
cosine_distances = [1 - sim for sim in cosine_similarities]

self.similarities = np.array(cosine_distances.copy())